### Read data 

In [ ]:
from datasets import load_dataset
data = load_dataset("uit-nlp/vietnamese_students_feedback")

In [ ]:
data#['train']

In [ ]:
read_data(data)

In [ ]:
data['train']['sentence'][12]

In [ ]:
train_data = data['train']#['train'][12]
#train_data['sentence']=pd.DataFrame(train_data['sentence']) 


In [ ]:
train_data

In [ ]:
data['train']['sentence'][12]

### 1. Lower case

In [ ]:
def lower_case(text):
    return text.lower()
    
# Apply the lower_case function to the 'sentence' column
train_data = train_data.map(lambda example: {'sentence': lower_case(example['sentence'])})

### 2. Transform emoticons to word form

In [1]:
from _2_0_dictionary import emotion2wordform_dict, wordform2vnese_dict, mispelling_dict, translate_dict, number_dict

def emoticon2word(sentence):
    words = sentence.split()
    converted_words = [emotion2wordform_dict.get(word, word) for word in words]
    converted_sentence = ' '.join(converted_words)
    return converted_sentence

# Test the function
sentence = "giao viên ko cóa dạy :)"
print("Converted sentence:", emoticon2word(sentence))

Converted sentence: giao viên ko cóa dạy colonsmile


### 3. Remove punctuation

In [5]:
import string

def remove_punctuation(input_string):
    # Create a translation table to remove all punctuation characters
    translator = str.maketrans('', '', string.punctuation)
    
    # Use the translate method to remove punctuation
    cleaned_string = input_string.translate(translator)
    
    return cleaned_string

print(remove_punctuation('thầy , cô dạy tốt . Trường ok.'))
#train_data = train_data.map(lambda example: {'sentence': remove_punctuation(example['sentence'])})

thầy  cô dạy tốt  Trường ok 


### 4. Translate word form of emoticons into VNese

In [10]:
def word_form2Vnese(sentence):
    words = sentence.split()
    print(words)
    converted_words = [wordform2vnese_dict.get(word, word) for word in words]
    converted_sentence = ' '.join(converted_words)
    return converted_sentence

# Test the function
sentence = "giao_viên ko cóa dạy colonsmile"
print("Converted sentence:", word_form2Vnese(sentence))

['giao_viên', 'ko', 'cóa', 'dạy', 'colonsmile']
Converted sentence: giao_viên ko cóa dạy cười nhẹ


### 5. Transform number to text

In [4]:
# Test cùng với "chấm thầy 10 điểm"
def number2text(sentence):
    words = sentence.split()
    converted_words = [number_dict.get(word, word) for word in words]
    converted_sentence = ' '.join(converted_words)
    return converted_sentence

# Test the function
sentence = "chấm thầy 10 điểm" 
print("Converted sentence:", number2text(sentence))

Converted sentence: chấm thầy mười điểm


### 6. Spelling correction, acronyms

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("wonrax/phobert-base-vietnamese-sentiment", use_fast=False)

sentences = "giáo_viên" # nhận thấy token khác nhau, 
#test cùng với độ tự tin khi dự đoán, nếu input sai chính tả 
inputs = tokenizer(sentences, padding= True, truncation=True),
inputs

In [ ]:
# Your dictionary tiếp cận 1 
# temp_dict = {
#     'không': ['ko', 'k', 'khum', 'khom', 'khom'],
#     'có': ['cóa', 'cóo', 'coá'],
#     'giảng viên': ['gv'], 
#     'giáo_viên' : ['gíao_viên', 'giao_viên']
# }

# def misspelling2true(sentence, dictionary):
#     words = sentence.split()
#     converted_words = [convert_list_in_dict(word, dictionary) for word in words]
#     converted_sentence = ' '.join(converted_words)
#     return converted_sentence

# def convert_list_in_dict(word, dictionary):
#     for key, values in dictionary.items():
#         if word in values:
#             return key
#     return word

# # Test the function with a sentence
# sentence_to_convert_back =  'giao viên này dạy ko hay như gv khác'
# converted_sentence_back = misspelling2true(sentence_to_convert_back, temp_dict)
# print("Converted sentence back:", converted_sentence_back)

In [9]:
#tiếp cận 2 
# dịch từ sai chính tả thành đúng chính tả
def mispell2word(sentence, dictionary = mispelling_dict):
    sentence = " " + sentence.strip() + " "
    for key, value_list in dictionary.items():
        for value in value_list:
            sentence = sentence.replace(value, key)
    return sentence

sentence = 'gv dạy hoc sinh trong phỏng máy k de hieu xíu nào'
print("Converted sentence back:", mispell2word(sentence))

Converted sentence back:  giảng viên dạy  học sinh  trong phòng máy không dễ hiểu xíu nào 


In [11]:
# dịch những từ tiếng anh thành tiếng việt 
def translate2word(sentence, dictionary = translate_dict):
    sentence = " " + sentence.strip() + " "
    for key, value_list in dictionary.items():
        for value in value_list:
            sentence = sentence.replace(value, key)
    return sentence

sentence = 'cô dạy max hay , rất pro , đưa bài lên web'
print("Converted sentence back:", translate2word(sentence))

Converted sentence back:  cô dạy cực kì hay , rất đỉnh , đưa bài lên trang mạng 


### 7. Remove extra whitespace

In [ ]:
def remove_extra_whitespace(input_string):
    words = input_string.split()
    return ' '.join(words)

# Example usage
input_string = "This    is      a   string     with     extra     spaces."
print(remove_extra_whitespace(input_string))  # Output: "This is a string with extra spaces."

### 8. Remove number (digits)

In [ ]:
train_data['sentence'][12]

In [ ]:
def remove_numbers(input_string):
    # Use the isalpha() method to filter out numeric characters
    cleaned_string = ''.join(char for char in input_string if not char.isdigit())
    return cleaned_string

train_data = train_data.map(lambda example: {'sentence': remove_numbers(example['sentence'])})

In [ ]:
train_data['sentence'][12]

### 9. Word Segmentation

In [ ]:
# thêm token bình thường
import underthesea
def word_segment(text):
    return underthesea.word_tokenize(text, format="text")

word_segment('gíao viên này dạy ko hay như gv khác')
#word_segment("Ông Nguyễn Khắc Chúc  đang làm việc tại Đại học Quốc gia Hà Nội. Bà Lan, vợ ông Chúc, cũng làm việc tại đây.")

In [ ]:
train_data = train_data.map(lambda example: {'sentence': word_segment(example['sentence'])})

In [ ]:
train_data['sentence']

### 10. Remove stopwords and particular words 

In [ ]:
def remove_stopwords(input_text, stopwords_file='stopword.txt'):
    # Read the custom stop words from the file
    with open(stopwords_file, 'r', encoding='utf-8') as file:
        stopwords = set(line.strip() for line in file)

    cleaned_words = [word for word in input_text.split() if word.lower() not in stopwords]
    cleaned_text = ' '.join(cleaned_words)

    return cleaned_text

# Test the function
input_text = "thầy rất tận tình và đi dạy rất đúng giờ"
cleaned_text = remove_stopwords(input_text, 'stopword.txt')
print(cleaned_text)

In [ ]:
train_data = train_data.map(lambda example: {'sentence': remove_stopwords(example['sentence'])})

### 11. Key_clause extraction

In [ ]:
import underthesea

def key_clause_extraction_vietnamese(text):
    sentences = underthesea.sent_tokenize(text)
    
    key_clauses = []
    for sentence in sentences:
        # Loại bỏ các câu không chứa thông tin (ví dụ: các câu chỉ chứa một số từ liên kết hoặc từ giới thiệu)
        if len(sentence.strip()) > 5:
            # Lưu lại câu chính đầu tiên (có thể mở rộng hơn để lựa chọn nhiều câu chính hơn)
            key_clauses.append(sentence)
            
    return key_clauses

# Ví dụ văn bản đầu vào (tiếng Việt)
text = "Trí tuệ nhân tạo (AI) là mô phỏng quá trình trí tuệ của con người bởi máy móc, đặc biệt là các hệ thống máy tính. Các quá trình này bao gồm học hỏi (tiếp thu thông tin và quy tắc để sử dụng thông tin đó), lập luận (sử dụng quy tắc để đạt đến kết luận gần đúng hoặc xác định) và tự sửa lỗi. AI có thể được phân loại thành AI yếu và AI mạnh. AI yếu, còn được gọi là AI hẹp, là một hệ thống AI được thiết kế và huấn luyện cho một nhiệm vụ cụ thể. Trợ lý cá nhân ảo, chẳng hạn như Siri của Apple, là một dạng AI yếu. AI mạnh, còn được gọi là trí tuệ nhân tạo tổng quát, là một hệ thống AI có khả năng tổng quát hóa như trí tuệ nhân tạo của con người. Khi được đưa ra một nhiệm vụ không quen thuộc, một hệ thống AI mạnh có thể tìm ra giải pháp mà không cần sự can thiệp của con người."
#text ="giảng viên phải nên cho ví dụ cụ thể những phần quan trọng nên nói kỹ có thể lấy ví dụ minh họa tránh việc lập đi lập lại nhiều lần sẽ tạo cảm giác chán cho sinh viên ."

# Trích xuất key clause
key_clauses = key_clause_extraction_vietnamese(text)

# In kết quả
for i, clause in enumerate(key_clauses, start=1):
    print(f"Key Clause {i}: {clause}")

In [ ]:
from _2_1_pre_processing import data_preprocessing

In [ ]:
train_data['sentence'][:20]

In [ ]:
train_data = train_data.map(lambda example: {'sentence': data_preprocessing(example['sentence'])})

In [ ]:
train_data['sentence'][:20]